In [ ]:
import geopandas as gpd

from ecodonut.utils import graph_to_gdf

rivers = gpd.read_file('example_data/rivers.gpkg')
water = gpd.read_file('example_data/water.gpkg')

In [ ]:
from ecodonut.river_graph import construct_water_graph

water_graph = construct_water_graph(rivers, water)

In [ ]:
import networkx as nx

nx.write_gml(water_graph, 'water_graph.gml', stringizer=lambda x: str(x))

In [ ]:
import pickle
import networkx as nx

pickle.dump(water_graph, open('water_graph.pickle', 'wb'))


In [ ]:
import pickle
import networkx as nx

water_graph = pickle.load(open('example_data/water_graph_78.pkl', 'rb'))

In [ ]:
for i, data in water_graph.nodes(data=True):
    water_graph.nodes(data=True)[i]['x'] = float(data['x'])
    water_graph.nodes(data=True)[i]['y'] = float(data['y'])

In [ ]:
nx.write_gml(water_graph, 'water_graph.gml', stringizer=lambda x: str(x))

In [ ]:
%load_ext autoreload
%autoreload 2
from ecodonut.river_graph import simulate_spill
import geopandas as gpd

# Симуляция разлива от gdf по направленному графу
industrial = gpd.read_parquet('example_data/industrial_lo_spb.parquet')

spill = simulate_spill(industrial, water_graph, min_dist_to_river=250, return_only_polluted=True)
spill

In [ ]:
spill.to_file('water_spill_industrial_new.gpkg')

In [ ]:
m = spill.explore(column='remain', cmap='autumn_r', tiles='CartoDB positron')
industrial[industrial['dist'] <= 100].explore(m=m, color='purple')
m.save('water_graph.html')

In [ ]:
lo = gpd.read_file('example_data/lo.geojson').to_crs(32636)
spill2 = spill.clip(lo)
industrial2 = industrial.clip(lo)

In [ ]:
# spill2.to_file('water_spill_lo3.gpkg')
industrial2[industrial2['dist'] <= 250].to_file('water_spill_industrial.gpkg')